# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
#%pip install faiss-cpu > /dev/null
#%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
#vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {}) # For future

vector_store = InMemoryVectorStore(OpenAIEmbeddings())

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

# 1. Create PROMPT template and chain
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

#Then, the SerpAPIWrapper is used for the Search tool. 
# It's crucial to explain that this allows the agent to perform web searches, which is vital for real-time information
search = SerpAPIWrapper()

# 2. Tool Definitions
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

# 3. Prompt Engineering
# The prefix and suffix in the prompt template set the context for the agent.
#  The prefix tells the agent to consider the objective and previous tasks, while the suffix structures the input.
#  The input variables are placeholders that get filled during execution.
# agent_scratchpad - Space for agent's reasoning process
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

# 4. Agent Prompt Construction
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [ ]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [12]:
OBJECTIVE = "Write a air temperature report for SF today"

In [13]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations= 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vector_store,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
first = baby_agi({"objective": OBJECTIVE})

In [ ]:
OBJECTIVE = "Write a wave condition report for PR today. Please write a summary of the wave condition report."
res = baby_agi({"objective": OBJECTIVE})

## Changing promt and tools 

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

# 1. Create a creative writing prompt template and chain
story_prompt = PromptTemplate.from_template(
    "You are a creative storyteller who weaves compelling narratives. Create an engaging story incorporating these elements: {objective}"
)
story_chain = LLMChain(llm=OpenAI(temperature=0.7), prompt=story_prompt)

search = SerpAPIWrapper()

# 2. Tool Definitions - Adding creative writing and research tools
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for researching facts, events, and inspiration for your story",
    ),
    Tool(
        name="CreateStory",
        func=story_chain.run,
        description="useful for generating creative stories and narratives. Input: story elements or theme to incorporate. Output: an engaging story that weaves these elements together.",
    ),
]

# 3. Prompt Engineering
# The prefix and suffix guide the AI to be more creative and narrative-focused
prefix = """You are an AI storyteller tasked with creating an engaging narrative around: {objective}. Consider these previous story elements for continuity: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

# 4. Agent Prompt Construction
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

llm = OpenAI(temperature=0.7)  # Higher temperature for more creativity
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

OBJECTIVE = "Create an adventure story about a time-traveling scientist in San Francisco. Please don't make it longer than one page."

# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations=2
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

baby_agi({"objective": OBJECTIVE})